In [0]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import confusion_matrix,classification_report
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D,Flatten, Reshape
from keras.datasets import fashion_mnist
from keras.models import Model
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras import backend as k
from sklearn.mixture import GaussianMixture as GMM
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mode

In [0]:
# Loads the training and test data sets from Keras
(images_train, labels_train), (images_test, labels_test) = fashion_mnist.load_data()
images_train=images_train.reshape(images_train.shape[0],images_train.shape[1]*images_train.shape[2])
images_test=images_test.reshape(images_test.shape[0],images_test.shape[1]*images_test.shape[2])

#Normalizing the Data by dividing the data by 255 (Maximum value of a pixel)
images_train = images_train.astype('float32') / 255
images_test = images_test.astype('float32') / 255

In [0]:
#Initializing KMeans object and Using Kmeans for Clustering
kmeans = KMeans(n_clusters=10,max_iter=500)
kmeans.fit(images_train)
y_pred_test=kmeans.predict(images_test)

In [0]:
#Creating Confustion Matrix and Heat Map
cm=confusion_matrix(labels_test,y_pred_test)
j=0
print(cm)
plt.figure(figsize=(8,8))
#Aligning Confusion MAtrix 
for i in range(10):
    maximum=np.max(cm[i])
    s=np.where(cm==maximum)
    cm[s[0],s[1]]=cm[i,j]
    cm[i,j]=maximum
    j=j+1
sns.heatmap(cm, annot=True,square=True,fmt="d",cbar=False, center=0,linewidth=1,cmap="Blues", xticklabels=np.unique(labels_train), yticklabels=np.unique(labels_train))
print(cm)
plt.xlabel('true Label')
plt.ylabel('predicted Label')
plt.show()

In [0]:
#Calculating Accuracy Score
accuracy_score_P1=metrics.normalized_mutual_info_score(labels_test,y_pred_test)
print(accuracy_score_P1*100)

In [0]:
#Preprocessing Testing data for Auto Encoder
images_train_cnn = images_train.reshape((len(images_train), 28, 28, 1))

#Preprocessing Testing data for Auto Encoder
images_test_cnn= images_test.reshape((len(images_test), 28, 28, 1))


In [0]:
#Building the Auto-Encoder Model using a CNN
autoencoder = Sequential()

# Encoder Layer
autoencoder.add(Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=images_train_cnn.shape[1:]))
autoencoder.add(MaxPooling2D((2, 2), padding='same'))
autoencoder.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
autoencoder.add(MaxPooling2D((2, 2), padding='same'))
autoencoder.add(Conv2D(8, (3, 3), strides=(2,2), activation='relu', padding='same'))

# Flatten Intermediate Part
autoencoder.add(Flatten())
autoencoder.add(Reshape((4, 4, 8)))

# Decoder Layer
autoencoder.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
autoencoder.add(UpSampling2D((2, 2)))
autoencoder.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
autoencoder.add(UpSampling2D((2, 2)))
autoencoder.add(Conv2D(16, (3, 3), activation='relu'))
autoencoder.add(UpSampling2D((2, 2)))
autoencoder.add(Conv2D(1, (3, 3), activation='sigmoid', padding='same'))

autoencoder.summary()

In [0]:
#Only Encoder Part so we can use this part for encoding the Training and Test Dataset before sending it to K-means clustering
only_encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('flatten_1').output)
only_encoder.summary()

In [0]:
#Compiling and Training the Encoder Model
autoencoder.compile(optimizer='adam', loss='mse',metrics=['accuracy'])
history=autoencoder.fit(images_train_cnn, images_train_cnn,epochs=25,validation_split=0.1,batch_size=500)

In [0]:
#Enocoding the Training and Test Data Set using only encoder part of the model
encoded_train_data = only_encoder.predict(images_train_cnn)
encoded_test_data = only_encoder.predict(images_test_cnn)


In [0]:
#Using K-means for clusturing the Encoded Images and then calculating the accuracy
kmeans = KMeans(n_clusters=10)
y_pred_train_cnn=kmeans.fit(encoded_train_data)
y_pred_test_P2=kmeans.predict(encoded_test_data)
training_score=metrics.normalized_mutual_info_score(labels_test,y_pred_test_P2)
print(training_score)

In [0]:
#Creating Confustion Matrix and Heat Map for Auto Encoder with K-means Clustering
cm_k=confusion_matrix(labels_test,y_pred_test_P2)
j=0
print(cm_k)
plt.figure(figsize=(8,8))
#Aligning Confusion MAtrix 
for i in range(10):
    maximum=np.max(cm_k[i])
    s=np.where(cm_k==maximum)
    cm_k[s[0],s[1]]=cm_k[i,j]
    cm_k[i,j]=maximum
    j=j+1
sns.heatmap(cm, annot=True,square=True,fmt="d",cbar=False, center=0,linewidth=1,cmap="Blues", xticklabels=np.unique(labels_train), yticklabels=np.unique(labels_train))
print(cm_k)
plt.xlabel('true Label')
plt.ylabel('predicted Label')
plt.show()

In [0]:
#Plotting graph of training loss and validation loss vs number of epochs while training for autoencoder
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [0]:
#Using Gaussian Mixture Model for clusturing the Encoded Images and then calculating the accuracy
gmm=GMM(n_components=10).fit(encoded_train_data)
y_pred_test_P3=gmm.predict(encoded_test_data)
training_score_gmm=metrics.normalized_mutual_info_score(labels_test,gmm_predict)
print(training_score_gmm)

In [0]:
#Creating Confustion Matrix and Heat Map for Auto Encoder with Gaussian Mixture Model
cm_GMM=confusion_matrix(labels_test,y_pred_test_P3)
j=0
print(cm_GMM)
plt.figure(figsize=(8,8))
#Aligning Confusion MAtrix with Cluster Ids
for i in range(10):
    maximum=np.max(cm_GMM[i])
    s=np.where(cm_GMM==maximum)
    cm_GMM[s[0],s[1]]=cm_GMM[i,j]
    cm_GMM[i,j]=maximum
    j=j+1
sns.heatmap(cm, annot=True,square=True,fmt="d",cbar=False, center=0,linewidth=1,cmap="Blues", xticklabels=np.unique(labels_train), yticklabels=np.unique(labels_train))
print(cm_GMM)
plt.xlabel('true Label')
plt.ylabel('predicted Label')
plt.show()


In [0]:
K.clear_session()